# Código 06: Circulo

Para formar o circulo, usaremos primitivas de triângulo. Para isso, usaremos em torno de 32 triângulos. As principais mudanças é na parte da elaboração do vetor de vértices.

## Código pré loop principal

### Inicialização do glfw e criação da janela

In [258]:
#Bibliotecas
!pip install glfw
import glfw
!pip install pyopengl
from OpenGL.GL import *
import OpenGL.GL.shaders #Não é redundante?
!pip install numpy
import numpy as np

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
#WIDTH_WINDOW : int = 800
WIDTH_WINDOW : int = 2000
#HEIGHT_WINDOW : int = 300
HEIGHT_WINDOW : int = 400
TITLE : str = "Circulo"
window = glfw.create_window(WIDTH_WINDOW, HEIGHT_WINDOW, TITLE, None, None)
glfw.make_context_current(window)

### Eventos de teclado e mouse

In [259]:
def key_event(window,key,scancode,action,mods):
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    
glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
    
glfw.set_mouse_button_callback(window,mouse_event)

### Shaders: código, espaço, compilação e linkagem

In [260]:
#GLSL para Vertex Shader
vertex_code = """
        attribute vec2 position;
        void main(){
            gl_Position = vec4(position,0.0,1.0);
        }
        """
#GLSL para Fragment Shader
fragment_code = """
        void main(){
            gl_FragColor = vec4(0.0, 0.0, 0.0, 1.0);
        }
        """

#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

### Criação dos vértices

* cos(angulo) = x/R
* sen(angulo) = y/R

In [261]:
def glsl_to_glfw(x_glsl, y_glsl):
    #Primeiro é necessário transformar o sistema glsl para variar entre número positivos
    #Neste caso, faremos variar de -1 a 1 para 0 a 2
    x_glsl_shiftado = x_glsl + 1.0
    y_glsl_shiftado = 1.0 - y_glsl

    #Segundo, aplicamos regra de três: x_glsl_shiftado/x_glfw = 2/WIDHT_WINDOW
    x_glfw = x_glsl_shiftado * (WIDTH_WINDOW/2)
    y_glfw = y_glsl_shiftado * (HEIGHT_WINDOW/2)
    return x_glfw, y_glfw

In [262]:
import math
import random

#Pi: valor constante
PI : float = 3.14
#Número de vértices: define a qualidade do circulos
num_vertices = 32
#Cria espaço para os vértices
vertices = np.zeros(num_vertices, [("position", np.float32, 2)])
#Raio do circulo que queremos criar
aspect_radius_x = 1
aspect_radius_y = 1

if WIDTH_WINDOW > HEIGHT_WINDOW: 
    aspect_radius_x = HEIGHT_WINDOW / WIDTH_WINDOW
else:
    aspect_radius_y = WIDTH_WINDOW / HEIGHT_WINDOW

raio = 0.5
#Variável auxiliar para calcular os 32 vértices
angulo = 0.0
#Centro da circunferência
c_x = random.uniform(-0.5, 0.5)
c_y = random.uniform(-0.5, 0.5)

for counter in range(num_vertices):
    #Variação do angulo em 32 vezes
    angulo += 2*PI/num_vertices 
    #Cálculos dos valores de (x,y)
    x = math.cos(angulo)*raio*aspect_radius_x + c_x
    y = math.sin(angulo)*raio*aspect_radius_y + c_y
    x_, y_ =  glsl_to_glfw(x,y)
    #vertices["position"][counter] = [x_,y_]
    vertices["position"][counter] = [x,y]

vertices

array([([ 0.3815192 ,  0.40937597],), ([ 0.3530947 ,  0.878143  ],),
       ([ 0.3069346 ,  1.310199  ],), ([ 0.24481098,  1.6889572 ],),
       ([ 0.16910885,  1.9998767 ],), ([ 0.08273442,  2.2310212 ],),
       ([-0.01099631,  2.373517  ],), ([-0.10848498,  2.4218934 ],),
       ([-0.20598894,  2.3742933 ],), ([-0.29976496,  2.2325442 ],),
       ([-0.3862129 ,  2.0020878 ],), ([-0.46201396,  1.6917715 ],),
       ([-0.52425814,  1.3135085 ],), ([-0.5705558 ,  0.8818206 ],),
       ([-0.5991296 ,  0.41328058],), ([-0.6088825 , -0.07412412],),
       ([-0.59944016, -0.5616817 ],), ([-0.571165  , -1.0306745 ],),
       ([-0.52514255, -1.4630975 ],), ([-0.4631397 , -1.8423499 ],),
       ([-0.3875367 , -2.153872  ],), ([-0.301236  , -2.385704  ],),
       ([-0.20755078, -2.528946  ],), ([-0.11007763, -2.5780985 ],),
       ([-0.01255864, -2.531275  ],), ([ 0.0812624 , -2.3902729 ],),
       ([ 0.16778363, -2.160505  ],), ([ 0.24368347, -1.8507929 ],),
       ([ 0.30604804, -1.473026  ]

### Manipulação dos espaços de dados

In [263]:
#Requisitando espaço de buffer para GPU
buffer = glGenBuffers(1)
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer)
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
#glBindBuffer(GL_ARRAY_BUFFER, buffer)

#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

### Exibindo na tela

In [264]:
glfw.show_window(window)

## Loop principal

* `GL_TRIANGLE_FAN`: nesta primitiva, os triângulos são formados conectando o primeiro vértice (o vértice central) a todos os outros vértices subsequentes na ordem em que eles são fornecidos. Isso cria um "leque" de triângulos que irradiam do vértice central.

In [265]:
while not glfw.window_should_close(window):
    glfw.poll_events()

    glPolygonMode(GL_FRONT_AND_BACK,GL_LINE) 
    glClear(GL_COLOR_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    #Primeiro número: vértice inicial
    #Segundo número: quantidade de vértices
    glDrawArrays(GL_TRIANGLE_FAN, 0, len(vertices))

    glfw.swap_buffers(window)

glfw.terminate()